In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import time
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

import tensorflow as tf
import spacy

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
train_clean = pd.read_csv('./train_clean.csv')
train_clean.head()

,Unnamed: 0,question_text,target,clean_text
0,0,How did Quebec nationalists see their province...,0,quebec nationalist see province nation
1,1,"Do you have an adopted dog, how would you enco...",0,adopted dog would encourage people adopt shop
2,2,Why does velocity affect time? Does velocity a...,0,velocity affect time velocity affect space geo...
3,3,How did Otto von Guericke used the Magdeburg h...,0,otto von guericke used magdeburg hemisphere
4,4,Can I convert montra helicon D to a mountain b...,0,convert montra helicon mountain bike changing ...


In [ ]:
# embdedding setup
# Source https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
embeddings_index = {}
EMBEDDING_FILE = './glove.42B.300d.txt' # define embedding file path

with open(EMBEDDING_FILE, 'r', encoding="utf-8") as f:
  for line in tqdm(f):
      values = line.split(" ")
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

print('Found %s word vectors.' % len(embeddings_index))

1917495it [01:56, 16476.84it/s]

Found 1917495 word vectors.


In [ ]:
train_df, val_df = train_test_split(train_clean.drop('clean_text',axis=1), test_size=0.07)
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (30 - len(embeds))
    return np.array(embeds)

# train_vects = [text_to_array(X_text) for X_text in tqdm(train_df["question_text"])]
val_vects = np.array([text_to_array(X_text) for X_text in tqdm(val_df["question_text"][:3000])])
val_y = np.array(val_df["target"][:3000])

100%|██████████| 3000/3000 [00:00<00:00, 18872.32it/s]


In [ ]:
# Data providers
batch_size = 128

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])

In [ ]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(30, 300)))
model.add(SpatialDropout1D(0.1))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 30, 128)          187392    
 l)                                                              
                                                                 
 spatial_dropout1d (SpatialD  (None, 30, 128)          0         
 ropout1D)                                                       
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              99328     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 286,849
Trainable params: 286,849
Non-trainable params: 0
__________________________________________________

In [ ]:
mg = batch_gen(train_df)
model.fit(mg, epochs=30,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

Epoch 1/30
1000/1000 [==============================] - 26s 15ms/step - loss: 0.1511 - accuracy: 0.9442 - val_loss: 0.1473 - val_accuracy: 0.9430
Epoch 2/30
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1358 - accuracy: 0.9478 - val_loss: 0.1444 - val_accuracy: 0.9430
Epoch 3/30
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1321 - accuracy: 0.9491 - val_loss: 0.1417 - val_accuracy: 0.9417
Epoch 4/30
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1289 - accuracy: 0.9505 - val_loss: 0.1452 - val_accuracy: 0.9463
Epoch 5/30
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1265 - accuracy: 0.9510 - val_loss: 0.1330 - val_accuracy: 0.9497
Epoch 6/30
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1263 - accuracy: 0.9504 - val_loss: 0.1331 - val_accuracy: 0.9457
Epoch 7/30
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1247 - accuracy: 0.9513 - val_loss: 0.1349 -

In [ ]:
from tensorflow.keras.saving import load_model
model.save('./LSTM.h5')

In [ ]:
model = load_model('./LSTM.h5')

In [ ]:
test_text = np.array([text_to_array("Which babies are more sweeter to their parents? Dark skin babies or light skin babies?")])

In [ ]:
test_text

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.33160999,  0.040621  , -0.43906   , ...,  0.3249    ,
          0.1026    ,  0.18494   ],
        [-0.32144001,  0.11529   ,  0.009374  , ..., -0.28773001,
          0.22002999,  0.19564   ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]])

In [ ]:
all_predict = model.predict(test_text).flatten()
y_te = (np.array(all_predict) > 0.5).astype(np.int)
y_te
all_predict

1/1 [==============================] - 0s 438ms/step


<ipython-input-72-e05452cfa1ce>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_te = (np.array(all_predict) > 0.5).astype(np.int)


array([0.5705933], dtype=float32)

In [ ]:
y_te

array([1])